In [487]:
import re
import os
import pandas as pd
import pdfplumber

In [488]:
pdf_path = "/Users/shivangsingh/Downloads/PFCM 2021-1 Servicer Report - Jan 25.pdf"
folder_name = "abbsr_files"
# pages_to_extract = [2, 5]

In [ ]:
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[1]  # Page index 1 for Page 2
    text = page.extract_text()  # Extract text instead of table

lines = text.split("\n")

for line in lines:
    print(line)


PRODIGY FINANCE CM 2021-1, DAC POOL SUMMARY
Distribution Date 27-Jan-2025 Collateral Strats
Collection Period End 07-Jan-2025
1 Collateral Strats Current Reporting Previous month 7th June 2021 CutOff (Closing)
All balances principal, admin fee and grace interest USD Loans USD Loans USD Loans
unless stated otherwise USD USD USD
Unique Borrowers Count (excluding Settled borrowers) 2 ,100 2 ,139 6 ,045
Loan Count (excluding Settled loans) 2 ,746 2 ,799 8 ,048
Principal Balance & Admin Fee Only 8 0,910,548 8 2,485,588 2 89,319,514
Principal Balance, Admin Fee & Interest During Grace 8 2,206,663 8 3,813,608 3 03,449,769
Average Loan Size incl Interest during Grace 2 9,937 2 9,944 3 7,705
WA Margin Loans (%) 6.61% 6.61% 6.56%
Lowest Margin (%) 4.50% 4.50% 4.50%
Last Maturity Date 07 July 2043 07 July 2043 07 July 2043
WA Original Repay Period (Years) 1 4.3 1 4.3 1 3.7
WA Remaining Term to Maturity (Years) 1 0.0 1 0.1 1 2.9
WA Remaining Grace Period (of Grace loans) (Years) - - 0 .42
WA Seaso

In [490]:

key_value_pairs = [
    ("PRODIGY FINANCE CM 2021-1, DAC", "POOL SUMMARY"),
    ("Distribution Date", lines[1].split()[2]),  # Extract the date
    ("Collection Period End", lines[2].split()[3])  # Extract the date
]

# Convert the key-value pairs into a DataFrame
df1 = pd.DataFrame(key_value_pairs, columns=["Key", "Value"])
df1
output_filename = os.path.join(folder_name, 'extracted_df1.xlsx')
df1.to_excel(output_filename, index=False, header=False)

In [ ]:

def fix_number_spacing(text):
    text = text.replace(",", "")  # Remove all commas
    
    # Fix number spacing except for dates and percentages
    text = re.sub(r'(\d{1,2}) (January|February|March|April|May|June|July|August|September|October|November|December) (\d{4})', r'\1_\2_\3', text)
    text = re.sub(r'(\d) (\d{3,})\b', r'\1\2', text)  # Fix thousands/millions
    text = re.sub(r'(\d) (\d\.\d+)', r'\1\2', text)  # Fix decimal numbers like 1 4.3 → 14.3
    text = re.sub(r'(\d) \.(\d+)', r'\1.\2', text)  # Fix cases like '9 .9' → '9.9'
    
    return text

def merge_split_lines(lines):
    """Merge lines starting with 'Period' into the previous line."""
    merged_lines = []
    
    for line in lines:
        if line.strip().startswith("Period"):
            # Merge with the last line in the list
            if merged_lines:
                merged_lines[-1] += " " + line.strip()  # Append it to the last line
        else:
            merged_lines.append(line.strip())  # Add new line as normal
    
    return merged_lines

def merge_loan_balance_lines(lines):
    """Merge lines starting with 'Total Modified Loan balance over Annual Period' with the next line if it starts with a number."""
    merged_lines = []
    i = 0

    while i < len(lines):
        line = lines[i].strip()

        # Check if the current line contains the target phrase
        if "Total Modified Loan balance over Annual Period" in line and i + 1 < len(lines):
            next_line = lines[i + 1].strip()
            
            # Check if the next line starts with a number
            if re.match(r'^\d', next_line):
                line += " " + next_line  # Merge the next line with the current line
                i += 1  # Skip the next line as it's merged

        merged_lines.append(line)
        i += 1

    return merged_lines

def handle_last_line(line):
    """Fix the last line where 'Modified Month' is incorrectly spaced and numbers are joined."""
    # Fix the spaced-out "Modified Month" issue
    # print("line", line)
    line = line.replace(",", "")
    line = re.sub(r'\bMod i f i e d\b', 'Modified', line)
    line = re.sub(r'\bM o n t h\b', 'Month', line)
    line = line.strip()
    # print("line", line.strip())
    parts = line.rsplit(" ", 2)  # Split by last 3 spaces
    # print("parts", parts)
    if len(parts) == 4 and parts[-2].isdigit() and parts[-1].isdigit():
        first_part, num1, num2 = parts[0], parts[-2], parts[-1]
        
        return f"{first_part} {num1} {num2}"  # Format properly
    
    return line + " "

merged_lines = merge_split_lines(lines)
merged_lines = merge_loan_balance_lines(merged_lines)
cleaned_lines = [fix_number_spacing(line) for line in merged_lines[4:-1]]

if "Total Ever since" in lines[-1]:
    lines[-1] = handle_last_line(lines[-1])
    cleaned_lines.append(lines[-1])

# cleaned_lines



['All balances principal admin fee and grace interest USD Loans USD Loans USD Loans',
 'unless stated otherwise USD USD USD',
 'Unique Borrowers Count (excluding Settled borrowers) 2100 2139 6045',
 'Loan Count (excluding Settled loans) 2746 2799 8048',
 'Principal Balance & Admin Fee Only 80910548 82485588 289319514',
 'Principal Balance Admin Fee & Interest During Grace 82206663 83813608 303449769',
 'Average Loan Size incl Interest during Grace 29937 29944 37705',
 'WA Margin Loans (%) 6.61% 6.61% 6.56%',
 'Lowest Margin (%) 4.50% 4.50% 4.50%',
 'Last Maturity Date 07_July_2043 07_July_2043 07_July_2043',
 'WA Original Repay Period (Years) 14.3 14.3 13.7',
 'WA Remaining Term to Maturity (Years) 10.0 10.1 12.9',
 'WA Remaining Grace Period (of Grace loans) (Years) - - 0.42',
 'WA Seasoning (Months in Repayment) 53.7 52.7 11.1',
 'Permitted maximum Modified Loan balance per Annual Period 4579435 4579435 15172488',
 'Total Modified Loan balance over Annual Period 30120 - -',
 'Modifie

In [492]:
data = []
for line in cleaned_lines:
    parts = line.rsplit(" ", 3)  # Split into last 3 columns + description
    if len(parts) == 4:
        desc, num1, num2, num3 = parts
        data.append([desc.strip(), num1.strip(), num2.strip(), num3.strip()])
    else:
        data.append([line, "", "", ""])  # Handle cases where format is different


df2 = pd.DataFrame(data, columns=["Collateral Strats", "Current Reporting", "Previous Month", "7th June 2021 CutOff"])
output_filename = os.path.join(folder_name, 'extracted_df2.xlsx')
df2.to_excel(output_filename, index=False, header=True)

In [493]:
# Page 5 extraction
with pdfplumber.open(pdf_path) as pdf:
    page = pdf.pages[4]  # Page index 1 for Page 2
    text = page.extract_text()  # Extract text instead of table

lines_page2 = text.split("\n")

for line in lines_page2:
    print(line)


2 Delinquency & Payment Arrangement Data
Annualised Constant Prepayment Rate 15.37%
Annualised Constant Default Rate 3.74%
Cumulative Default rate 5.98%
All balances are principal, admin fee and grace interest, unless stated otherwise
Pool Balance 6 7,195,566
defined as (Agg Loans - defaulted loans)
USD
Delinquency - Total Pool Outstanding Principal # Loans % of Principal WA seasoning WA margin
a Current 5 8,670,917.7 2,098 71.4% 5 4.0 6.5%
b Delinquent (1 instalment) 3 ,757,421.5 130 4.6% 5 4.0 6.6%
c Delinquent (2 instalments) 1 ,958,806.8 55 2.4% 5 6.5 6.9%
d Delinquent (3 instalments) 1 ,384,310.8 39 1.7% 5 1.5 6.8%
e Delinquent (4 instalments) 5 35,755.8 19 0.7% 5 9 6.8%
f Delinquent (5 instalments) 6 35,453.1 14 0.8% 5 3 7.0%
g Delinquent (6 instalments) 2 52,900.2 7 0.3% 5 0 6.6%
h Default (7+ instalments) 1 5,011,097.4 384 18.3% 5 2.3 6.9%
Grand Total 8 2,206,663.5 2 ,746 100% 5 3.7 6.61%
Delinquency - Forbearance Outstanding Principal # Loans % of Principal WA seasoning WA mar

In [494]:
pool_balance = lines_page2[5].replace(",", "").split()
pool_balance = pool_balance[-2]+ pool_balance[-1]

key_value_pairs = [
    ("Delinquency & Payment Arrangement Data", " "),
    ("Annualised Constant Prepayment Rate", lines_page2[1].split()[-1]),  # Extract the date
    ("Annualised Constant Default Rate", lines_page2[2].split()[-1]),  # Extract the date
    ("Cumulative Default rate", lines_page2[3].split()[-1]),
    ("All balances are principal, admin fee and grace interest, unless stated otherwise", ""),
    ("Pool Balance", pool_balance),
    (lines_page2[6], lines_page2[7])
]

# Convert the key-value pairs into a DataFrame
df3 = pd.DataFrame(key_value_pairs, columns=["Key", "Value"])
output_filename = os.path.join(folder_name, 'extracted_df3.xlsx')
df3.to_excel(output_filename, index=False, header=False)

In [ ]:

def fix_number_spacing(text):
    # Remove commas
    text = text.replace(",", "")
    def remove_spaces_from_last(text):

        words = text.split()
        if len(words) >= 3 and words[-2] != '-' and words[-3] != '-':
            words[-3] = words[-3] + words[-2]  
            words.pop(-2) 
        
        if len(words) >= 6 and words[-2] != '-' and words[-3] != '-':
            words[-6] = words[-6] + words[-5]  
            words.pop(-5)  
        
        return " ".join(words)
    
    return remove_spaces_from_last(text)



cleaned_lines = [fix_number_spacing(line) for line in lines_page2[9:]]
cleaned_lines

['a Current 58670917.7 2098 71.4% 54.0 6.5%',
 'b Delinquent (1 instalment) 3757421.5 130 4.6% 54.0 6.6%',
 'c Delinquent (2 instalments) 1958806.8 55 2.4% 56.5 6.9%',
 'd Delinquent (3 instalments) 1384310.8 39 1.7% 51.5 6.8%',
 'e Delinquent (4 instalments) 535755.8 19 0.7% 59 6.8%',
 'f Delinquent (5 instalments) 635453.1 14 0.8% 53 7.0%',
 'g Delinquent (6 instalments) 252900.2 7 0.3% 50 6.6%',
 'h Default (7+ instalments) 15011097.4 384 18.3% 52.3 6.9%',
 'Grand Total 8 2206663.52 746 100% 53.7 6.61%',
 'Delinquency - Forbearance Outstanding Principal # Loans %of Principal WA seasoningWA margin',
 'Current 5 28875.81 3 0.6% 55.5 6.8%',
 'Delinquent (1 instalment) 40646.9 2 0.0% 61.0 8.0%',
 'Delinquent (2 instalments) 187367.9 5 0.2% 46.2 7.3%',
 'Delinquent (3 instalments) 82407.4 2 0.1% 43.6 4.7%',
 'Delinquent (4 instalments) - - 0.0% - -',
 'Delinquent (5 instalments) - - 0.0% - -',
 'Delinquent (6 instalments) - - 0.0% - -',
 'Default (7+ instalments) 250308.7 3 0.3% 55.5 5.9

In [ ]:
def remove_spaces_from_first(text):

        words = text.split()
        
        if len(words) >= 6:
            # Check if both words at -6 and -5 are numeric
            if words[-6].replace('.', '', 1).isdigit() and words[-5].replace('.', '', 1).isdigit():
                words[-6] = words[-6] + words[-5]  # Concatenate as a string
                words.pop(-5)  # Remove the now unnecessary element

        
        return " ".join(words)

cleaned_lines_final = [remove_spaces_from_first(line) for line in cleaned_lines]
cleaned_lines_final

['a Current 58670917.7 2098 71.4% 54.0 6.5%',
 'b Delinquent (1 instalment) 3757421.5 130 4.6% 54.0 6.6%',
 'c Delinquent (2 instalments) 1958806.8 55 2.4% 56.5 6.9%',
 'd Delinquent (3 instalments) 1384310.8 39 1.7% 51.5 6.8%',
 'e Delinquent (4 instalments) 535755.8 19 0.7% 59 6.8%',
 'f Delinquent (5 instalments) 635453.1 14 0.8% 53 7.0%',
 'g Delinquent (6 instalments) 252900.2 7 0.3% 50 6.6%',
 'h Default (7+ instalments) 15011097.4 384 18.3% 52.3 6.9%',
 'Grand Total 82206663.52 746 100% 53.7 6.61%',
 'Delinquency - Forbearance Outstanding Principal # Loans %of Principal WA seasoningWA margin',
 'Current 528875.81 3 0.6% 55.5 6.8%',
 'Delinquent (1 instalment) 40646.9 2 0.0% 61.0 8.0%',
 'Delinquent (2 instalments) 187367.9 5 0.2% 46.2 7.3%',
 'Delinquent (3 instalments) 82407.4 2 0.1% 43.6 4.7%',
 'Delinquent (4 instalments) - - 0.0% - -',
 'Delinquent (5 instalments) - - 0.0% - -',
 'Delinquent (6 instalments) - - 0.0% - -',
 'Default (7+ instalments) 250308.7 3 0.3% 55.5 5.9%'

In [497]:
def process_numbers(text):
    # Find all numbers in the text (including decimals and percentages)
    words = text.split()  # Split text into words

    for i in range(len(words)):
        # Match numbers with more than 1 decimal place (excluding percentages)
        if re.match(r'^\d+\.\d{2,}$', words[i]):  
            original_number = words[i]  # Store the original number as a string
            
            rounded_number = "{:.1f}".format(float(original_number))  # Keep one decimal place as a string
            
            extra_value = str(int(round((float(original_number) - float(rounded_number)) * 100, 0)))  # Extract decimal part as string
            
            words[i] = rounded_number  # Replace with rounded number
            
            # Find the next valid number (excluding percentages) and add extra_value
            for j in range(i+1, len(words)):
                if re.match(r'^\d+(\.\d+)?$', words[j]):  # Ensure it's a valid number (not %)
                    words[j] = str(extra_value) + words[j]
                    break  # Stop after modifying the next number

    return " ".join(words) 

cleaned_lines_finall = [process_numbers(line) for line in cleaned_lines_final]
cleaned_lines_finall

['a Current 58670917.7 2098 71.4% 54.0 6.5%',
 'b Delinquent (1 instalment) 3757421.5 130 4.6% 54.0 6.6%',
 'c Delinquent (2 instalments) 1958806.8 55 2.4% 56.5 6.9%',
 'd Delinquent (3 instalments) 1384310.8 39 1.7% 51.5 6.8%',
 'e Delinquent (4 instalments) 535755.8 19 0.7% 59 6.8%',
 'f Delinquent (5 instalments) 635453.1 14 0.8% 53 7.0%',
 'g Delinquent (6 instalments) 252900.2 7 0.3% 50 6.6%',
 'h Default (7+ instalments) 15011097.4 384 18.3% 52.3 6.9%',
 'Grand Total 82206663.5 2746 100% 53.7 6.61%',
 'Delinquency - Forbearance Outstanding Principal # Loans %of Principal WA seasoningWA margin',
 'Current 528875.8 13 0.6% 55.5 6.8%',
 'Delinquent (1 instalment) 40646.9 2 0.0% 61.0 8.0%',
 'Delinquent (2 instalments) 187367.9 5 0.2% 46.2 7.3%',
 'Delinquent (3 instalments) 82407.4 2 0.1% 43.6 4.7%',
 'Delinquent (4 instalments) - - 0.0% - -',
 'Delinquent (5 instalments) - - 0.0% - -',
 'Delinquent (6 instalments) - - 0.0% - -',
 'Default (7+ instalments) 250308.7 3 0.3% 55.5 5.9%'

In [498]:
columns = ["Delinquency - Total Pool", "Outstanding Principal", "# Loans", "% of Principal", "WA seasoning", "WA margin"]

processed_data = []
for row in cleaned_lines_finall[:9]:
    words = row.split()  # Split by space
    name = " ".join(words[:-5])  # Join everything except the last 5 numbers
    values = words[-5:]  # Last 5 items are numeric values
    processed_data.append([name] + values)

# Create DataFrame
df4 = pd.DataFrame(processed_data, columns=columns)
output_filename = os.path.join(folder_name, 'extracted_df4.xlsx')
df4.to_excel(output_filename, index=False, header=True)

In [499]:
columns = ["Delinquency - Forbearance", "Outstanding Principal", "# Loans", "% of Principal", "WA seasoning", "WA margin"]

processed_data = []
for row in cleaned_lines_finall[10:]:
    words = row.split()  # Split by space
    name = " ".join(words[:-5])  # Join everything except the last 5 numbers
    values = words[-5:]  # Last 5 items are numeric values
    processed_data.append([name] + values)

# Create DataFrame
df5 = pd.DataFrame(processed_data, columns=columns)
df5 = df5.drop(df5.index[-1]) 
output_filename = os.path.join(folder_name, 'extracted_df5.xlsx')
df5.to_excel(output_filename, index=False, header=True)

In [ ]:
output_file = 'prodigy_abbsr_pdf_output.xlsx'
tables_folder = os.path.join(os.getcwd(), "abbsr_files")
excel_files = [f for f in os.listdir(tables_folder) if f.endswith('.xlsx')]
sorted_file_names = sorted(excel_files, key=lambda x: int(re.search(r'(\d+)', x).group()))
# sorted_file_names
with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
    start_row = 0  # Track the starting row for each dataset
    
    for file in sorted_file_names:
        # Read the data from the current file
        file_path = os.path.join(folder_name, file)
        # print("file_path", file_path)
        data = pd.read_excel(file_path)
        
        # Write data to the final Excel file at the correct row position
        data.to_excel(writer, sheet_name='Sheet1', startrow=start_row, index=False)

        # Update start_row to place the next dataset after 2 empty rows
        start_row += len(data) + 2 

file_path abbsr_files/extracted_df1.xlsx
file_path abbsr_files/extracted_df2.xlsx
file_path abbsr_files/extracted_df3.xlsx
file_path abbsr_files/extracted_df4.xlsx
file_path abbsr_files/extracted_df5.xlsx
